# 환경설정

In [3]:
# 설치(import)
import pytesseract
import cv2

# TesseractOCR

In [4]:
# 경로 설정 (본인 경로에 맞게 확인)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

img = cv2.imread('./test.png')
# dst = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # 색상 변환이 필요할 경우 사용
text = pytesseract.image_to_string(img, lang='kor+eng')

print(text)

[MAI 85) 【[ 국 세 청 민 원 사 무 처 리 규 정 제 18 호 서 식 】

Bows AL ey 7] 2h
즉 시

스 A 느 소
느
OF 소 또 거 소

( 법 인 은 본 점 소 재 지 ) | Sara Bae Aa. 7 、 서 엉 윤 마 늘 _ 03

@ 사 업 장 소 제 지 | 겠 라 북 되 얀 성시 M22 . 179/| Oa 9 !

@ 상 호 ( 법 인 패 샵 주 식 호 | 삭

@) 사 업 자 등 록 번 호 47 _ |38042|<@ 전 화 번 호 10 ㅣ (28 7/67

이 하 운 @ 주 민 ( Yel) 등 록 번 호 | 3500 고 ㅇ 3295/94

Al TA @ 수 량 97

위 납 세 자 의 위 임 을 받 아 대 리 인이 신 청 및 수 령 하 는 경 우 에 는 아래 사 항 을 기 재 하 여 주 시기 바 랍 니 다 .

대 리 인 3 주 민 등 록 번 호 전 화 번 호 남 세 자 와 의 관 계
5905 [0 3392984 | 032 908 3196 강석

이 러 하 다

위 의 내 용 이 사 실과 같 음 을 증 명 하 여 주 시기 바 랍 니 다 .
1991 년 09 월 [8 일
신 청 인 ( 본 인 )
대 리 인

세 무 서 장 귀 하
※ 대 리인이 신 청 할 BS. 민 원 서 류 위 임 장 [ 국 세 청 민 원 사 무 처 리 규 정 ALS MAS 작 성 하 여 제 출
하 여 야 합 니 다 .

※ 개 인 정 보 보 호 법 제 24 조 에 의 한 수 짐 . 이 용 동 의 | 신 청 인 ( 본 인 )]

O 수 집 . 이 용 목 적 ( 증 명 발 급 , 사 후 관 리 등 ) ㅇ 보 유 . 이 용 기 간 ( 5 년 )

O 수 짐 대상 고 유 식 별 정 보 [ 주 민 등 록 번 호 , 외 국 인 등 록 번 호 , 여 권 번 호 )

= 상 기 내 용 에 대 해 동 의 함 [, 동 의 하 지 않 음

O 동 의 를 거 부 할 권 리 가 있 으 며 , 동 의 거 부 에 따 라 불 이 익 ( 증 명 발 급 불 가 )이 있 을 수 있 음
위 와